## 1) Weather model
## For this assignment, you'll revisit the historical temperature dataset. To complete this assignment, submit a link a Jupyter notebook containing your solutions to the following tasks:

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn import linear_model

from sqlalchemy import create_engine

import warnings

warnings.filterwarnings('ignore')

## First, load the dataset from the weatherinszeged table from Thinkful's database.

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

weath_df = pd.read_sql_query('select * from weatherinszeged',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

## Investigate the data

In [3]:
weath_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96453 entries, 0 to 96452
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   date                 96453 non-null  datetime64[ns, UTC]
 1   summary              96453 non-null  object             
 2   preciptype           96453 non-null  object             
 3   temperature          96453 non-null  float64            
 4   apparenttemperature  96453 non-null  float64            
 5   humidity             96453 non-null  float64            
 6   windspeed            96453 non-null  float64            
 7   windbearing          96453 non-null  float64            
 8   visibility           96453 non-null  float64            
 9   loudcover            96453 non-null  float64            
 10  pressure             96453 non-null  float64            
 11  dailysummary         96453 non-null  object             
dtypes: datetime64[ns, 

In [4]:
weath_df.head(10)

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.
5,2006-04-01 03:00:00+00:00,Partly Cloudy,rain,9.222222,7.111111,0.85,13.9587,258.0,14.9569,0.0,1016.66,Partly cloudy throughout the day.
6,2006-04-01 04:00:00+00:00,Partly Cloudy,rain,7.733333,5.522222,0.95,12.3648,259.0,9.9820,0.0,1016.72,Partly cloudy throughout the day.
7,2006-04-01 05:00:00+00:00,Partly Cloudy,rain,8.772222,6.527778,0.89,14.1519,260.0,9.9820,0.0,1016.84,Partly cloudy throughout the day.
8,2006-04-01 06:00:00+00:00,Partly Cloudy,rain,10.822222,10.822222,0.82,11.3183,259.0,9.9820,0.0,1017.37,Partly cloudy throughout the day.
9,2006-04-01 07:00:00+00:00,Partly Cloudy,rain,13.772222,13.772222,0.72,12.5258,279.0,9.9820,0.0,1017.22,Partly cloudy throughout the day.


In [5]:
weath_df.describe()

,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure
count,96453.000000,96453.000000,96453.000000,96453.000000,96453.000000,96453.000000,96453.0,96453.000000
mean,11.932678,10.855029,0.734899,10.810640,187.509232,10.347325,0.0,1003.235956
std,9.551546,10.696847,0.195473,6.913571,107.383428,4.192123,0.0,116.969906
min,-21.822222,-27.716667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,4.688889,2.311111,0.600000,5.828200,116.000000,8.339800,0.0,1011.900000
50%,12.000000,12.000000,0.780000,9.965900,180.000000,10.046400,0.0,1016.450000
75%,18.838889,18.838889,0.890000,14.135800,290.000000,14.812000,0.0,1021.090000
max,39.905556,39.344444,1.000000,63.852600,359.000000,16.100000,0.0,1046.380000


## Like in the previous checkpoint, build a linear regression model where your target variable is the difference between the apparenttemperature and the temperature. As explanatory variables, use humidity and windspeed.

In [6]:
weath_df['temp_diff'] = weath_df['apparenttemperature'] - weath_df['temperature']
weath_df['temp_diff']

0       -2.083333
1       -2.127778
2        0.000000
3       -2.344444
4       -1.777778
           ...   
96448    0.000000
96449    0.000000
96450    0.000000
96451    0.000000
96452    0.000000
Name: temp_diff, Length: 96453, dtype: float64

## Now, estimate your model using OLS. What are the R-squared and adjusted R-squared values? Do you think they are satisfactory? Why?

In [7]:
# Y is the target variable
y = weath_df['temp_diff']

# X is the feature set
X = weath_df[['humidity','windspeed']]

# We add constant to the model as it's a best practice
# to do so every time!
X = sm.add_constant(X)

# We fit an OLS model using statsmodels
results = sm.OLS(y, X).fit()

# We print the summary results.
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              temp_diff   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Wed, 09 Sep 2020   Prob (F-statistic):               0.00
Time:                        03:48:08   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.0

* R-squared is 0.288, and adjusted R-squared is 0.288
* These are terrible values, they should be well over 0.70

## Next, include the interaction of humidity and windspeed to the model above and estimate the model using OLS. Now, what is the R-squared of this model? Does this model improve upon the previous one?

In [8]:
y = weath_df['temp_diff']

weath_df['humid_wind'] = weath_df['humidity'] * weath_df['windspeed']

X = weath_df[['humidity','windspeed', 'humid_wind']]

# We add constant to the model as it's a best practice
# to do so ever ytime!
X = sm.add_constant(X)

# We fit an OLS model using statsmodels
results = sm.OLS(y, X).fit()

# We print the summary results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              temp_diff   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Wed, 09 Sep 2020   Prob (F-statistic):               0.00
Time:                        03:48:08   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0839      0.033      2.511      0.0

* New model did improve the R-squared from 0.29 to 0.34

## Add visibility as an additional explanatory variable to the first model and estimate it. Did R-squared increase? What about adjusted R-squared? Compare the differences put on the table by the interaction term and the visibility in terms of the improvement in the adjusted R-squared. Which one is more useful?

In [9]:
# Y is the target variable
y = weath_df['temp_diff']

# X is the feature set
X = weath_df[['humidity','windspeed', 'visibility']]

# We add constant to the model as it's a best practice
# to do so every time!
X = sm.add_constant(X)

# We fit an OLS model using statsmodels
results = sm.OLS(y, X).fit()

# We print the summary results.
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              temp_diff   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.303
Method:                 Least Squares   F-statistic:                 1.401e+04
Date:                Wed, 09 Sep 2020   Prob (F-statistic):               0.00
Time:                        03:48:08   Log-Likelihood:            -1.6938e+05
No. Observations:               96453   AIC:                         3.388e+05
Df Residuals:                   96449   BIC:                         3.388e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5756      0.028     56.605      0.0

* R-squared and adjusted R-squared did improve from 0.29 to 0.30
* interaction term was more useful at 0.34

## Choose the best one from the three models above with respect to their AIC and BIC scores. Validate your choice by discussing your justification with your mentor.

* Second model had the best AIC and BIC with 3.334e+05

------

## 2. House prices model
## In this exercise, you'll work on your house prices model. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

## Load the houseprices data from Thinkful's database.

In [10]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

house_df = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

## Run your house prices model again and assess the goodness of fit of your model using F-test, R-squared, adjusted R-squared, AIC and BIC.

In [11]:
house_df = house_df[house_df['saleprice'] < 300000]

In [12]:
X = house_df[['lotarea', 'overallcond', 'yearbuilt']]
y = house_df['saleprice']

X = sm.add_constant(X)

results = sm.OLS(y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.418
Model:                            OLS   Adj. R-squared:                  0.417
Method:                 Least Squares   F-statistic:                     321.1
Date:                Wed, 09 Sep 2020   Prob (F-statistic):          4.30e-157
Time:                        03:48:09   Log-Likelihood:                -16152.
No. Observations:                1345   AIC:                         3.231e+04
Df Residuals:                    1341   BIC:                         3.233e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -2.083e+06   7.85e+04    -26.522      0.000   -2.24e+06   -1.93e+06
lotarea         1.6510      0.136     12.099      0.000       1.383       1.919
overallcond  8707.8409   1032.643      8.433      0.000    6682.070    1.07e+04
yearbuilt    1108.0041     38.753     28.592      0.000    1031.982    1184.026
==============================================================================
Omnibus:                       87.460   Durbin-Watson:                   1.945
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              124.294
Skew:                           0.544   Prob(JB):                     1.02e-27
Kurtosis:                       4.016   Cond. No.                     9.31e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.31e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Do you think your model is satisfactory? If so, why?

* R-squared values are not good, but the F-test values are very good

## In order to improve the goodness of fit of your model, try different model specifications by adding or removing some variables.

In [15]:
house_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1345 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1345 non-null   int64  
 1   mssubclass     1345 non-null   int64  
 2   mszoning       1345 non-null   object 
 3   lotfrontage    1095 non-null   float64
 4   lotarea        1345 non-null   int64  
 5   street         1345 non-null   object 
 6   alley          91 non-null     object 
 7   lotshape       1345 non-null   object 
 8   landcontour    1345 non-null   object 
 9   utilities      1345 non-null   object 
 10  lotconfig      1345 non-null   object 
 11  landslope      1345 non-null   object 
 12  neighborhood   1345 non-null   object 
 13  condition1     1345 non-null   object 
 14  condition2     1345 non-null   object 
 15  bldgtype       1345 non-null   object 
 16  housestyle     1345 non-null   object 
 17  overallqual    1345 non-null   int64  
 18  overallc

In [19]:
X = house_df[['lotarea', 'overallcond', 'yearbuilt', 'totalbsmtsf', 'garagearea']]
y = house_df['saleprice']

X = sm.add_constant(X)

results = sm.OLS(y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.564
Model:                            OLS   Adj. R-squared:                  0.562
Method:                 Least Squares   F-statistic:                     346.2
Date:                Wed, 09 Sep 2020   Prob (F-statistic):          3.05e-238
Time:                        03:51:54   Log-Likelihood:                -15958.
No. Observations:                1345   AIC:                         3.193e+04
Df Residuals:                    1339   BIC:                         3.196e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -1.429e+06    7.5e+04    -19.051      0.000   -1.58e+06   -1.28e+06
lotarea         0.9054      0.124      7.318      0.000       0.663       1.148
overallcond  8771.5395    895.718      9.793      0.000    7014.376    1.05e+04
yearbuilt     746.1681     37.813     19.733      0.000     671.988     820.348
totalbsmtsf    32.0563      2.697     11.885      0.000      26.765      37.348
garagearea     75.5516      5.560     13.589      0.000      64.645      86.459
==============================================================================
Omnibus:                      166.778   Durbin-Watson:                   1.941
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1616.288
Skew:                          -0.116   Prob(JB):                         0.00
Kurtosis:                       8.365   Cond. No.                     1.03e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.03e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## For each model you try, get the goodness of fit metrics and compare your models with each other. Which model is the best and why?

In [22]:
X = house_df[['lotarea', 'overallcond', 'yearbuilt', 'totalbsmtsf', 'garagearea', 'fullbath']]
y = house_df['saleprice']

X = sm.add_constant(X)

results = sm.OLS(y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.641
Model:                            OLS   Adj. R-squared:                  0.639
Method:                 Least Squares   F-statistic:                     397.3
Date:                Wed, 09 Sep 2020   Prob (F-statistic):          5.19e-293
Time:                        03:54:31   Log-Likelihood:                -15828.
No. Observations:                1345   AIC:                         3.167e+04
Df Residuals:                    1338   BIC:                         3.171e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -1.085e+06   7.11e+04    -15.257      0.000   -1.22e+06   -9.45e+05
lotarea         0.7507      0.113      6.659      0.000       0.530       0.972
overallcond  9447.0413    814.472     11.599      0.000    7849.259     1.1e+04
yearbuilt     550.3089     36.246     15.182      0.000     479.203     621.415
totalbsmtsf    29.7478      2.453     12.125      0.000      24.935      34.561
garagearea     62.1451      5.111     12.158      0.000      52.118      72.172
fullbath     3.123e+04   1848.863     16.893      0.000    2.76e+04    3.49e+04
==============================================================================
Omnibus:                      189.522   Durbin-Watson:                   1.901
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2215.027
Skew:                          -0.164   Prob(JB):                         0.00
Kurtosis:                       9.278   Cond. No.                     1.07e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.07e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""